### This pipline is mainly for the following OpenVINO models:

* person-detection-0200
* person-detection-0201
* person-detection-0202
* person-detection-0203

In [1]:
import cv2
import time
import threading
import openvino as ov
import numpy as np

In [2]:
class VideoPlayer:

    def __init__(self, source, fps=None):

        self.cv2 = cv2  # This is done to access the package in class methods
        self.__cap = cv2.VideoCapture(source)
        if not self.__cap.isOpened():
            raise RuntimeError(
                f"Cannot open {'camera' if isinstance(source, int) else ''} {source}"
            )
        # fps of input file
        self.__input_fps = self.__cap.get(cv2.CAP_PROP_FPS)
        if self.__input_fps <= 0:
            self.__input_fps = 60
        # target fps given by user
        self.__output_fps = fps if fps is not None else self.__input_fps
        self.__size = None
        
        # first frame
        _, self.__frame = self.__cap.read()
        self.__lock = threading.Lock()
        self.__thread = None
        self.__stop = False

    def start(self):
        self.__stop = False
        self.__thread = threading.Thread(target=self.__run, daemon=True)
        self.__thread.start()

    def stop(self):
        self.__stop = True
        if self.__thread is not None:
            self.__thread.join()
        self.__cap.release()

    def __run(self):
        prev_time = 0
        while not self.__stop:
            t1 = time.time()
            ret, frame = self.__cap.read()
            if not ret:
                break

            # fulfill target fps
            if 1 / self.__output_fps < time.time() - prev_time:
                prev_time = time.time()
                # replace by current frame
                with self.__lock:
                    self.__frame = frame

            t2 = time.time()
            # time to wait [s] to fulfill input fps
            wait_time = 1 / self.__input_fps - (t2 - t1)
            # wait until
            time.sleep(max(0, wait_time))

        self.__frame = None

    def next(self):
        import cv2

        with self.__lock:
            if self.__frame is None:
                return None
            # need to copy frame, because can be cached and reused if fps is low
            frame = self.__frame.copy()
        return frame

In [3]:
class DetectionPipeline:
    def __init__(self, model_path, device='CPU'):
        self.core = ov.Core()
        self.model_path = model_path
        self.device = device
        self.model = self.load_model()
        self.compiled_model = self.core.compile_model(model=self.model, device_name=self.device)
        self.input_layer_ir = self.model.input(0)
        self.shape = self.get_shape()

    def load_model(self):
        model = self.core.read_model(model=self.model_path)
        return model

    def get_shape(self):
        N, C, H, W = self.input_layer_ir.shape
        return (H, W)
        
    def preprocess(self, image):
        resized_image = cv2.resize(image, self.shape)
        resized_image = cv2.cvtColor(np.array(resized_image), cv2.COLOR_BGR2RGB)
        resized_image = resized_image.transpose((2, 0, 1))
        resized_image = np.expand_dims(resized_image, axis=0).astype(np.float32)
        return resized_image

    def postprocess(self, result, image, fps):

        detections = result.reshape(-1, 7)
        for i, detection in enumerate(detections):
            _, image_id, confidence, xmin, ymin, xmax, ymax = detection
            if confidence > 0.5:
                xmin = int(max((xmin * image.shape[1]), 10))
                ymin = int(max((ymin * image.shape[0]), 10))
                xmax = int(min((xmax * image.shape[1]), image.shape[1] - 10))
                ymax = int(min((ymax * image.shape[0]), image.shape[0] - 10))
                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                cv2.putText(image, str(round(fps, 2)) + " fps", (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 3) 
        return image

In [4]:
def run_sync(source, required_fps):
    
    frame_number = 0
    required_fps = 30
    player = None
    title = "Output"

    infer_request = obj.compiled_model.create_infer_request()

    try:
        # Create a video player
        player = VideoPlayer(source, fps=required_fps)
        # Start capturing
        start_time = time.time()
        player.start()
        
        cv2.namedWindow(title, cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)
        
        while True:
            frame = player.next()
            if frame is None:
                print("Source ended")
                break
            
            resized_frame = obj.preprocess(frame)
            infer_request.set_tensor(obj.input_layer_ir, ov.Tensor(resized_frame))
            infer_request.infer()
            res = infer_request.get_output_tensor(0).data
            stop_time = time.time()
            total_time = stop_time - start_time
            frame_number = frame_number + 1
            sync_fps = frame_number / total_time

            frame = obj.postprocess(res, frame, sync_fps)

            
            cv2.putText(frame, str(round(sync_fps, 2)) + " fps", (5, 100), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 1)
            cv2.putText(frame, str(frame_number) + " frame", (5, 150), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 0), 1)
            
            cv2.imshow(title, frame)
            key = cv2.waitKey(1)
            # escape = 27
            if key == 27:
                break
            
    except KeyboardInterrupt:
        print("Interrupted")
    # Any different error
    except RuntimeError as e:
        print(e)
    finally:
        cv2.destroyAllWindows()
        if player is not None:
            player.stop()

In [5]:
def run_async(source, required_async_fps):
    
    frame_number = 0
    
    current_request = obj.compiled_model.create_infer_request()
    next_request = obj.compiled_model.create_infer_request()

    required_async_fps = 30
    player = None
    title = "Output"


    try:
        # Create a video player
        player = VideoPlayer(source, fps=required_async_fps)
        # Start capturing
        start_time = time.time()
        player.start()
        frame = player.next()
        resized_frame = obj.preprocess(frame)
        current_request.set_tensor(obj.input_layer_ir, ov.Tensor(resized_frame))
        current_request.start_async()
        cv2.namedWindow(title, cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)
        
        while True:
            next_frame = player.next()
            if next_frame is None:
                print("Source ended")
                break
            
            resized_frame = obj.preprocess(next_frame)

            next_request.set_tensor(obj.input_layer_ir, ov.Tensor(resized_frame))
            next_request.start_async()

            current_request.wait()

            res = current_request.get_output_tensor(0).data
            stop_time = time.time()
            total_time = stop_time - start_time
            frame_number = frame_number + 1
            async_fps = frame_number / total_time

            frame = obj.postprocess(res, frame, async_fps)

            
            cv2.putText(frame, str(round(async_fps, 2)) + " fps", (5, 100), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 1)
            cv2.putText(frame, str(frame_number) + " frame", (5, 150), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 0), 1)
            
            cv2.imshow(title, frame)
            key = cv2.waitKey(1)
            # escape = 27
            if key == 27:
                break
            
            frame = next_frame
            current_request, next_request = next_request, current_request
            
    except KeyboardInterrupt:
        print("Interrupted")
    # Any different error
    except RuntimeError as e:
        print(e)
    finally:
        cv2.destroyAllWindows()
        if player is not None:
            player.stop()

In [6]:
if __name__ == "__main__":
    model_path = "/home/acer/workspace/intel_models/intel/person-vehicle-bike-detection-2000/FP32/person-vehicle-bike-detection-2000.xml"
    obj = DetectionPipeline(model_path=model_path)
    run_sync(source=2, required_fps=30)
    run_async(source=2, required_async_fps=30)